# 1. Import libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# 2. Set global variables and import data sets 

### Why did we choose a small test set with 4000 observations?
In machine learning, we often train a model on a large training dataset and test it on a smaller amount of data as the larger the training set, the better the model learns. Among 54607 observations in the trainning set, we selected around 7% of the data to be used for testing which means that this model was trained on approximately 93% of the raw data resulting in a higher precision of model performance.

Another important reason is that we chose the test size of 4000 because that's about the same size as the kaggle private leaderboard test size
the public score is computed with about 2000 samples, and the private score is computed with about 4000 samples.


In [ ]:
seed = 977
n_splits = 17 # number of cross validation splits
test_size = 4000 

df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 3. Data preprocessing: feature engineering, scaling, and missing data imputation

**We changed the variable ‘education’ from being categorically encoded to numerically encoded.**
We transfered the categorical variable “education” into continuous variable representing the number of educational years.

In [ ]:
income = df["high_income"].to_numpy()
df.drop("high_income", axis=1, inplace=True)

df = pd.concat([df, test_df]) # concatenate train & test for feature transformation

enc_edu = {'Children': 0,
            'Less than 1st grade': 0,
            '1st 2nd 3rd or 4th grade': 1,
            '5th or 6th grade': 1,
            '7th and 8th grade': 1,
            '9th grade': 1,
            '10th grade': 1,   
            '11th grade': 1,
            '12th grade no diploma': 1,
            'High school graduate': 4,
            'Some college but no degree': 6,
            'Associates degree-occup /vocational': 7,
            'Associates degree-academic program': 7,
            'Bachelors degree(BA AB BS)': 8,
            'Masters degree(MA MS MEng MEd MSW MBA)': 10,
            'Doctorate degree(PhD EdD)': 16,
            'Prof school degree (MD DDS DVM LLB JD)': 16}

df["education"] = df['education'].map(enc_edu) # change categorical encoding to numerical

#### Why do we perform data preprocessing on the train and test sets together? Can this cause data leakage?

Leakage can occur when data from the test set is included in fitting scaling and/or imputing algorithms. Technically, models should be trained solely using data from the train set. This means that, when performing data preprocessing such as scaling or imputing, the scaler should be fit on the train set alone, and then applied to both the train and test sets. Leakage can lead to overly optimisitic results on the test set.

However, we did fit our scaling and imputing algorithms on both the train and test sets, possibly introducing leakage. We did this partly because it is a bit tricky to fit those algorithms on the train set alone, and then use them to transform the train and test sets separately. Ultimately, however, we justify this mainly because while we acknowledge that there may be some limited amount of feature leakage, there is no target leakage, because we remove the target variable from the train and test sets before performing data preprocessing on the features. 


#### Why do we use one-hot encoding instead of label encoding?

Not all algorithms can handle categorical predictors natively. Hence, it common practice to transform categorical features into numerical ones. There are a few methods that can be used for this process: one-hot encoding, label encoding, target encoding, and more. 

One-hot encoding creates a new feature for each category of a categorical feature. An observation is recorded as 1 if it falls into that category, and 0 otherwise. A disadvantage of this method is that it can really cause the dimensionality and sparsity of the data to blow up. This can cause algorithms that either do not handle sparsity well, or are particularly susceptible to the curse of dimensionality, to suffer in terms of performance.

Label encoding encodes each category with integers ranging from 0 to num_classes - 1. This can be disadvantageous when the categories are not ordinal. If the categories cannot be ordered, but nonetheless have been processed using label encoding, algorithms may learn a relationship between the hierarchy falsely implied by integer encoding and the target variable. 

We mainly rely on tree based learning algorithms in this project. Tree based learning algorithms can handle integer encoded data well, and several of them (such as CatBoost) can handle categorical data well with no need for one-hot encoding or integer encoding. Nevertheless, we will use one-hot encoding because despite issues with sparsity and high dimensionality because it will more easily allow us to compare tree based learning algorithms with other algorithms without having to perform data preprocessing separately for each. This may sound a bit lazy, but we thought that it would be pretty tough, to process the data one way for logistic regression, another way for random forests, and so on. 


#### What are missingness indicators and why do we add them?
 A missingness indicator (0/1) shows you which entries were missing before imputation. We added the missingness indicators because sometimes, there is a relationship between the "missingness" and the predictor label. This gives us also some possibly interesting information.

In [ ]:
df["isnull"] = df.isnull().sum(axis=1) # add column indicating the number of NaN entries per row (missingness indicator)

categorical_var = ["mig_chg_msa", "tax_filer_stat", "det_hh_summ", "mig_prev_sunbelt", 
          "hisp_origin", "vet_question", "country_self",
          "mig_move_reg", "hs_college", "class_worker", "mig_same",
          "unemp_reason", "state_prev_res", "race", "country_mother",
          "sex", "ind_code_level1", "citizenship", "union_member",
          "fam_under_18", "marital_stat", "region_prev_res", "mig_chg_reg",
          "country_father", "occ_code_level1", "full_or_part_emp", "det_hh_fam_stat"] # list of categorical variables

numerical_var = ["age", "stock_dividends", "wage_per_hour", "capital_losses",
           "capital_gains", "weeks_worked", "education"] # list of numerical variables
           
for feature in categorical_var:
    df.loc[df[feature].isna(), feature] = feature + "_missing" # replace NaN entries with a string
    dummies = pd.get_dummies(df[feature]) # create dummies
    dummies_columns = []
    for i in range(dummies.shape[1]):
        dummies_columns.append(feature + "_" + dummies.columns[i])
    dummies.columns = dummies_columns 
    df.drop(feature, axis = 1, inplace = True)
    df = pd.concat([df, dummies], axis=1)

for feature in numerical_var: # missingness indicator
    df[feature + "_missing"] = 0
    df.loc[df[feature].isna(), feature + "_missing"] = 1

**Some categories (eg. occ_code_level1 and occ_code_level2) contain more or less the same information but have different encodings. How did we deal with this and why?** (Shown in EDA)
We used one hot encoding for most of the categorical variables except occ_code_level2 as it comes already label encoded. Since those two variables contained more or less the same information, we kept the occ_code_level2 as it was. The same treatment was applied to ind_code_level1 and ind_code_level2. We kept the label encoded variables as they are because tree based algorithms work well with label encoded variables, and we wanted to avoid increasing the dimensionality of the data. 

**Why did we not use methods to reduce high dimensionality as a result of using one-hot encoding? For instance, combining categories that appear infrequently, or using PCA.**
As mentioned above, one-hot encoding represents categorical variables as binary vectors and therefore, using one-hot encoding increases the dimensionality of the data set as it creates a new variable for each level in the variable. Also, another issue with one-hot encoding is that it may lead to multicollinearity which is a huge problem with Logistic Regression. In conclusion, it is natural to think that we should reduce the dimentionality by using PCA or stepwise method. However, after applying those methods, they did not provide a higher accuracy score, so we decided to drop them in the final models. 



## What is IterativeImputer and how does it work? 
IterativeImputer is a strategy for imputing missing values by modeling each feature with missing values as a regression function of other features. It does so in an iterative way: at each step, a feature column is designated as output y and the other feature columns are treated as inputs X. A regressor is fit on (X, y) for known y. Then, the regressor is used to predict the missing values of y. This is done for each feature in an iterative fashion. It is repeated for max_iter imputation rounds with final results.

### Why did we choose it over KNNImputer?



KNNImputer imputes missing values with the mean value from the parameter ‘n_neighbors’ nearest neighbors found in the training set. 

We chose IterativeImputer over KNNImputer, because IterativeImputer takes into account correlations between features (if they exist) and will impute the missing value according to the correlation with the other features. Another reason is why we used ItreativeImputer is because it is much less computational expensive, compared to KNNImputer. 

Another reason why we chose to use IterativeImputer is that: In high dimensional data, we encounter higher occurrences of having a lot of missing values and a high correlation between a number of variables. Particularly, KNN Imputer runs based on the k-Nearest Neighbors algorithm which relates to distance computation and is highly affected by the variable correlation and missing values. In order to overcome these problems, one should apply KNN Imputer with the correct parameters: weights (Weight function used in prediction) and metric (Distance metric for searching neighbors). Therefore, it is safer to use IterativeImputer.

### How did we choose the parameters for IterativeImputer? 

**n_nearest_features** is the number of other features used to estimate the missing values of each feature column. To ensure that all features are used throughout the process, the neighbor features are not necessarily nearest, but are drawn with probability proportional to correlation for each imputed target feature. 

We choose for IterativeImputer(n_nearest_features = 7, random_state=seed) the given parameters, because specifying n_nearest_features gives a significant speed-up when the number of features is huge. Also, by running cross-validation using for example LGBMClassifier for several different values of n_nearest_features, n_nearest_features=7 gave the best results.

In [ ]:
df[df.columns] = StandardScaler().fit_transform(df[df.columns])
X_imputed = IterativeImputer(n_nearest_features = 7, random_state=seed).fit_transform(df.iloc[:, 1:])

X_train_imputed, X_test_imputed, y_train, y_test = train_test_split(X_imputed[:-test_df.shape[0], :], 
                                                                    income, 
                                                                    test_size=test_size, 
                                                                    random_state=seed,
                                                                    shuffle=True, 
                                                                    stratify=income)
X_kaggle_imputed = X_imputed[-test_df.shape[0]:,:]

# 4. Choosing Classifiers


## 4.1 Logistic regression

The first method that we applied for this dataset is Logistic regression due tp the nature of the binary response. **Logistic regression** is a  method to predict a binary outcome by calculating or predicting the probability (conditional probability) of a binary event occurring. It is a linear classification method as its decision boundary is linear. This method gives us an accuracy score of 0.8585. Here are some advantages and disadvantages of the method:

**Advantages**
- Easier to implement and interpret
- Can have the probability of an observation belongs to a group
- It performs well for simple datasets as well as when the data set is linearly separable.


**Disadvantages**
- There are several assumptions for a good logistic fit such as: The predicted outcome is strictly binary, variables are independent from each others (no multicollinearity) among the independent variables. If the training data does not satisfy the above assumptions, logistic regression may not work well
- Not all data are linear seperate




In [ ]:
model = LogisticRegression(tol=1e-5)
space = dict()
space['solver'] = ['liblinear']
space['penalty'] = ['l1']
space['C'] = [0.05, 0.06, 0.07]
search = GridSearchCV(model, space, scoring='accuracy', 
                      cv=StratifiedKFold(n_splits=10, random_state=seed, shuffle=True).split(X_train_imputed, y_train),
                      n_jobs=-1)
result = search.fit(X_train_imputed, y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.8584583457314832
Best Hyperparameters: {'C': 0.06, 'penalty': 'l1', 'solver': 'liblinear'}


## 4.2 Random Forest
The second model that we run was random forest. **Random Forest** is an extension of the concept of bagging. Instead of selecting all features for training the individual trees, here, random forest draws random subsets of variables to be used for the split. This subset of predictors changes for each split to avoid too similar trees which is a problem in bagging method.  

We chose the number of tree that we want to build are 100,200,300 and there is no constraint in how many processors is it allowed to use(n_jobs=-1). This method gave us the accuracy score of  0.8598614062586641 with the best n_estimators equal to 200. Here are some pros and cons of the method:

**Advantages**
- Random forest results in a great predictive performance, 
- It only has one tunning parameter which is the size of the subset of variables, 
- Can perform variable importance
- Instead of using cross validation to test the model's ability to predict new data, random forest uses out of bag estimator
- Dimensionality reduction as each tree does not consider all the attributes
- Each decision tree created is independent of the other hence gets risk of parallelization

**Disadvantages**
- It is easy to be see that random forest has a good predictive power but a bad interpretation power.
- Training time is longer compared to other models due to its complexity.


In [ ]:
model = RandomForestClassifier()
space = dict()
space['n_estimators'] = [100, 200, 300]
search = GridSearchCV(model, space, scoring='accuracy', 
                      cv=StratifiedKFold(n_splits=10, random_state=seed, shuffle=True).split(X_train_imputed, y_train),
                     n_jobs=-1)
result = search.fit(X_train_imputed, y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.8598614062586641
Best Hyperparameters: {'n_estimators': 200}


## 4.3 Gradient Boosting Classification

Machine Learning methods can be used to fit a model individually, but they can also be put together to an ensemble. As they get combined, they can create a more powerful model, as each method is fitted after another. In each step, they account for improving the shortcomings of the previous models and hence (in the long-term) improve the prediction error. This approach is called boosting.

**Gradient Boosting** is a type of boosting with a gradient algorithm. It starts with an individual decision tree and builds new decision trees based on the errors of the previous model. Each new model takes a step in the direction that minimizes the prediction error, in the space of possible predictions for each training sample. For a classification problem, we use the Gradient Boosting Classifier.


Explain important parameters. What is the learning rate? n_estimators? How does l1 regularization work?

The **learning rate alpha** defines how fast the model learns. As each tree added changes the overall model, alpha controls the magnitude of change. Low values of alpha indicate a slow learning process (but it makes the model robuster and more efficent), high values of alpha do the opposite.

**n_estimators** is the number of trees that are used in the model. If the learning rate is low, we need more trees to train the model. However, the choice of the number of trees is essential, because too many trees tend to overfit the model.


**Advantages**
- prediction speed and accuracy


**Disadvantages**
- slow to train on large data sets


What is **histogram-based gradient boosting**?

Because the training process of Gradient Boosting is generally slow, the training of the added trees can be done faster by discretizing (binning) the continuous input variables to a few hundred unique values. Gradient boosting ensembles that tailor the training algorithm around input variables under this transform are called histogram-based gradient boosting ensembles. 





## Different Algorithms for Gradient Boosting
In the following part, we want to present three different types of gradient boosting algorithms.

### 4.3.1 XGBoost Classifier

The Extreme Gradient Boosting (XGBoost) Classifier works in the framework of Gradient Boosting. It is originally written in C++, and additionally, the library is parallelizable which means the core algorithm can run on clusters of GPUs or even across a network of computers.

In [ ]:
model = XGBClassifier(objective='binary:logistic', eval_metric='logloss', tree_method = "hist", use_label_encoder=False)
space = dict()
space['n_estimators'] = [1000, 1100, 1200]
space['learning_rate'] = [0.03]
space['reg_alpha'] = [4, 5, 6]
search = GridSearchCV(model, space, scoring='accuracy', 
                      cv=StratifiedKFold(n_splits=10, random_state=seed, shuffle=True).split(X_train_imputed, y_train), 
                      n_jobs=-1)
cv_xgb = search.fit(X_train_imputed, y_train)
print('Best Score: %s' % cv_xgb.best_score_)
print('Best Hyperparameters: %s' % cv_xgb.best_params_)

Best Score: 0.8730016759955422
Best Hyperparameters: {'learning_rate': 0.03, 'n_estimators': 1100, 'reg_alpha': 5}


### 4.3.2 LightGBM Classifier
The Light Gradient Boosting Machines (LightGBM) is another version of Gradient Boosting, which was created by Microsoft. It is outstanding (in comparison to XGBoost and CatBoost) in terms of speed.

In [ ]:
model = LGBMClassifier()
space = dict()
space['n_estimators'] = [1100, 1200, 1300]
space['learning_rate'] = [0.03]
space['reg_alpha'] = [2, 3, 4]
search = GridSearchCV(model, space, scoring='accuracy', 
                      cv=StratifiedKFold(n_splits=10, random_state=seed, shuffle=True).split(X_train_imputed, y_train), 
                      n_jobs=-1)
cv_lgbm = search.fit(X_train_imputed, y_train)
print('Best Score: %s' % cv_lgbm.best_score_)
print('Best Hyperparameters: %s' % cv_lgbm.best_params_)

Best Score: 0.8731795923722678
Best Hyperparameters: {'learning_rate': 0.03, 'n_estimators': 1200, 'reg_alpha': 3}


### 4.3.3 CatBoost Classifier
CatBoost was developped by the russian company Yandex. It does not have the same popularity as XGBoost and LightGBM, but has several nice features, such as the easy treatment of categorical variables.

In [ ]:
model = CatBoostClassifier(n_estimators=1100, verbose=0)
space = dict()
space['l2_leaf_reg'] = [2, 3, 4]
space['max_depth'] = [7, 8, 9]
search = GridSearchCV(model, space, scoring='accuracy', 
                      cv=StratifiedKFold(n_splits=10, random_state=seed, shuffle=True).split(X_train_imputed, y_train), 
                      n_jobs=-1)
cv_catboost = search.fit(X_train_imputed, y_train)
print('Best Score: %s' % cv_catboost.best_score_)
print('Best Hyperparameters: %s' % cv_catboost.best_params_)

Best Score: 0.8735746969970315
Best Hyperparameters: {'l2_leaf_reg': 3, 'max_depth': 8}


**Why do we use multiple implementations of what is essentially the same thing?**

The algorithms are quite similar, but have differences mostly related to the splitting mechanism. 

**LightGBM**: uses gradient-based one-side sampling (GOSS) that selects the split using all the instances with large gradients (i.e., large error) and a random sample of instances with small gradients. In order to keep the same data distribution when computing the information gain, GOSS introduces a constant multiplier for the data instances with small gradients. Thus, GOSS achieves a good balance between increasing speed by reducing the number of data instances and keeping the accuracy for learned decision trees.

**CatBoost**: uses Minimal Variance Sampling (MVS), which is a weighted sampling version of Stochastic Gradient Boosting. In this technique, the weighted sampling happens in the tree-level and not in the split-level. The observations for each boosting tree are sampled in a way that maximizes the accuracy of split scoring.

**XGBoost**: is not using any weighted sampling techniques, which makes its splitting process slower compared to GOSS and MVS

The performance of XGBoost, LightGBM and CatBoost depends very much on the data set. Our expectation was that the (however present) differences are enough to provide enough diversity to make an ensemble stronger and more stable than any of the individual algorithms.



# 5. Hyperparamater tuning



**While tuning the gradient boosting algorithms, we set a specific learning rate and then tuned the other parameters. Why?**

We set a specific learning rate, because GB tends to quickly overfit the data. The learning rate slows it down a bit. Because of this, one can more easily find the right other parameters e.g. the number of trees such that the accuracy is highest.

**We didn’t use the 1-se rule. Why?**

It is tricky to use the 1 standard error rule and choosing the simplest model when tuning more than one parameter using GridsearchCV. Also, GridsearchCV is computationally costly when the algorithms take such a long time to run.

The principle of parsimony tells us that when increases the number of parmeters, the variance increases whereas the bias decreases. We skipped the 1-se rule and relied on the fact that ensembling models will also do the trick in terms of reducing variance.


# Train classifiers

In [ ]:
y_clf_test, y_clf_kaggle, classifiers, cross_val_pred = [[] for i in range(4)]

for classifier in [["Logistic Regression", LogisticRegression(solver='liblinear', random_state=seed, C = 0.05, penalty='l1')],
                   ["Random Forests", RandomForestClassifier(n_estimators=200, random_state=seed)],
                   ["XGBoost", XGBClassifier(n_estimators = 1100, learning_rate = 0.03, reg_alpha = 5, 
                           objective='binary:logistic', eval_metric='logloss', tree_method = "hist", use_label_encoder=False)],
                   ["LightGBM", LGBMClassifier(n_estimators = 1200, learning_rate = 0.03, reg_alpha = 3, random_state=seed)],
                   ["CatBoost", CatBoostClassifier(n_estimators = 1100, l2_leaf_reg = 3, max_depth = 8, verbose = 0, random_seed=seed)]]:
    
    cv = StratifiedKFold(n_splits=n_splits, random_state=seed, shuffle=True).split(X_train_imputed, y_train)
    cv_est = cross_validate(classifier[1], X_train_imputed, y_train, cv=cv, return_estimator=True, n_jobs=-1)
    cv = StratifiedKFold(n_splits=n_splits, random_state=seed, shuffle=True).split(X_train_imputed, y_train)
    cross_val_pred.append(cross_val_predict(classifier[1], X_train_imputed, y_train, cv=cv, method='predict_proba', n_jobs=-1)[:, 1])
    
    print(classifier[0] + " mean CV test score:", cv_est['test_score'].mean())
    print(classifier[0] + " std CV test score:", cv_est['test_score'].std())
    
    for clf in cv_est['estimator']:
        y_clf_test.append(clf.predict_proba(X_test_imputed)[:, 1])
        y_clf_kaggle.append(clf.predict_proba(X_kaggle_imputed)[:, 1])
        
    classifiers.append(classifier[1].fit(X_train_imputed, y_train)) # fit on the entire train set

Logistic Regression mean CV test score: 0.8587743886983423
Logistic Regression std CV test score: 0.00597475595212089
Random Forests mean CV test score: 0.8595843884922505
Random Forests std CV test score: 0.006198897107016615
XGBoost mean CV test score: 0.8733176483017492
XGBoost std CV test score: 0.005245000030098439
LightGBM mean CV test score: 0.8734956549124141
LightGBM std CV test score: 0.005481240489130057
CatBoost mean CV test score: 0.8735153146503545
CatBoost std CV test score: 0.0063768433725236605


## Which classifiers have the best cross-validation accuracy?
It can be seen that CatBoost, XGBoost and LightGBM have fairly high cross-validation accuracy (nearly 87,3%). The second rank belongs to logistic regression and random forests (around 86%).

# 6. Build meta-models

## What are ensemble methods? What are their advantages and disadvantages?
**Ensemble methods** are techniques that create multiple models and then combine them to produce improved results. Ensemble methods usually produces more accurate solutions than a single model would. Ensemble methods for classification can be named: voting, stacking, bagging and boosting, random forest, etc.

**Advantages:**
- Having higher predictive accuracy, compared to the individual models.
- Different models can be combined to handle types of data (linear or non-linear data).
- Bias/variance can be reduced 
- More stable compared to other models

**Disadvantages:**
- High preditive power with the cost of less interpretation.
- It is always a question of how to pick the right selection of models, the wrong one can lead to lower predictive accuracy than an individual model.
- Computationally expensive



## How did we choose what models to ensemble for the meta-models?
We tried out several combinations, but an ensemble of just the gradient boosted models worked best. 



## Meta-models : What is voting?

Voting includes a diverse collection of model types, such as a decision tree, logistic Regression, and many more. The model that combines all the predictions is called "Meta-Model" and the individual contributing models are called "Base-Models". In comparison to Stacking, no individual weights (based on their performance) are given to the models. Instead they are all assumed to contribute equally. In classification, predictions are made by averaging the predictions, then choosing the class with the most votes (hard voting) or the largest summed probability (soft voting). Soft voting is only recommended when the individual classifiers are well calibrated.

## Meta -models: What is stacking? What are its advantages and disadvantages?

Stacking is a framework and an ensemble method. It uses a meta-model to learn which ensemble members are reliable in order to combine predictions from contributing ensemble members in the best way. The way it works is the different base-models, each of them having their stregth and weaknesses, are fitted to the training data. Their predictions serve as features for the meta model, which gives the final classification.

**Advantages**
- makes predictions that have better performance than any single model in the ensemble
- improves accuracy while keeping variance and bias low


**Disadvantages**
- not even stacked ensembles are perfect and new observations can be predicted wrongly
- hard to interpret

**We did not include logistic regression and random forest in the stacking ensemble. Why?**
Basically, the predictions from logistic regression and random forest have much lower prediction accuracy than the gradient boosting algorithms, and are not diverse enough to strengthen the ensemble. In fact, including them in the ensemble actually seems to decrease the accuracy of the ensemble model.

In [ ]:
stack_X_train = np.vstack(cross_val_pred[2:]).T
stack_X_test = np.vstack([classifiers[i].predict_proba(X_test_imputed)[:,1] for i in range(2, 5)]).T
stack_X_kaggle = np.vstack([classifiers[i].predict_proba(X_kaggle_imputed)[:,1] for i in range(2, 5)]).T

model = LogisticRegression(solver='saga', penalty='l2', max_iter=1000, tol = 1e-5)
parameters = {'C': np.linspace(np.sqrt(0.001), np.sqrt(1), n_splits)**2}
search = GridSearchCV(model, parameters, scoring='accuracy', cv = n_splits, n_jobs=-1)
result = search.fit(stack_X_train, y_train)
print('Best CV accuracy:', result.best_score_)
print('Std:', result.cv_results_['std_test_score'][result.best_index_])

meta_model = LogisticRegression(solver='saga', penalty='l2', C = result.best_params_['C'], max_iter=1000, tol = 1e-5).fit(stack_X_train, y_train)

stack_test = meta_model.predict(stack_X_test)
soft_max_vote_test = np.where(np.sum(np.vstack(y_clf_test[(n_splits*2):]).T, axis=1)/(n_splits*3) <= 0.5, 0, 1) # soft max vote of cv estimators
hard_max_vote_test = np.where(np.sum(np.where(np.vstack(y_clf_test[(n_splits*2):]).T <= 0.5, 0, 1), axis=1) <= (n_splits*3)//2, 0, 1) # hard max vote of cv estimators

stack_kaggle = meta_model.predict(stack_X_kaggle)
soft_max_vote_kaggle = np.where(np.sum(np.vstack(y_clf_kaggle[(n_splits*2):]).T, axis=1)/(n_splits*3) <= 0.5, 0, 1)
hard_max_vote_kaggle = np.where(np.sum(np.where(np.vstack(y_clf_kaggle[(n_splits*2):]).T <= 0.5, 0, 1), axis=1) <= (n_splits*3)//2, 0, 1)

Best CV accuracy: 0.8733966952086616
Std: 0.005496146445086672


# 7. Model and meta-model performance and comparison

In [ ]:
print("Test accuracy of soft max_vote using the CV estimators:", (soft_max_vote_test == y_test).sum()/y_test.shape[0])
print("Test accuracy of hard max_vote using the CV estimators", (hard_max_vote_test == y_test).sum()/y_test.shape[0])
print("Test accuracy of model stacking:", meta_model.score(stack_X_test, y_test))
print("Test accuracy of Logistic Regression:", classifiers[0].score(X_test_imputed, y_test))
print("Test accuracy of Random Forest:", classifiers[1].score(X_test_imputed, y_test))
print("Test accuracy of XGBoost:", classifiers[2].score(X_test_imputed, y_test))
print("Test accuracy of LGBM:", classifiers[3].score(X_test_imputed, y_test))
print("Test accuracy of CatBoost:", classifiers[4].score(X_test_imputed, y_test))

Test accuracy of soft max_vote using the CV estimators: 0.8755
Test accuracy of hard max_vote using the CV estimators 0.8765
Test accuracy of model stacking: 0.87525
Test accuracy of Logistic Regression: 0.86225
Test accuracy of Random Forest: 0.86125
Test accuracy of XGBoost: 0.8745
Test accuracy of LGBM: 0.8765
Test accuracy of CatBoost: 0.87575


## Why couldn't we evaluate the performance of the meta-models using cross-validation?
One drawback of cross-validation is that it requires to fit K-1 different models and test on the k-th group, which can be computationally costly. Moreover, it is all about the question of what is the good number of K. Unfortunately, it depends on the size of the data and the choice of K is always a bias-variance tradeoff. Specifically, large K lead to small variance and high bias but small K leads to large bias and small variance. Therefore, the effect of randomness in cross-validation estimation is high.

Another reason is that it takes a lof of time to perform cross validation on meta-models.

# 8. Submission

## What model do we select for final submission? Why?
We select hard_max_vote for our final submission. We realize that it is methodologically unsound to choose a model whose performance has not been evaluated with more rigorous methods, such as cross-validation. It would be more sound to submit predictions made by XGBoost, LGBM, or CatBoost, for which we do have cross-validation results. However, we think that the voting ensemble is at least as strong as the individual learners that contribute to it, and hopefully more stable. 

In [ ]:
kaggle_submission = hard_max_vote_kaggle
kaggle_submission = np.vstack((test_df["id"].to_numpy(), kaggle_submission)).T
kaggle_submission = pd.DataFrame(kaggle_submission)
kaggle_submission.columns = ["id", "high_income"]
kaggle_submission.astype("int64").to_csv("hard_max_vote.csv", index=False)